<center>
<img src="logo.png" height="900"> 
</center>


#  Авокадо

Хорошо известно, что миллениалы любят тосты с авокадо. Хорошо известно, что миллениалы живут со своими родителями. На самом деле они не могут купить себе дом, потому что едят слишком много тостов с авокадо. Но надежда есть. Если нам удастся найти город с дешёвым авокадо, миллениалы смогут туда переехать и накопить денег на собственные квартиры. В этом задании мы этим и займёмся!

> __ВАЖНО!!!__ Перед отправкой тетрадки в грейдер на оценивание закомментируйте код, который обучает модели. Оставьте только ответы, записанные в соотвествующие переменные. При выставлении оценки ваш код должен отрабатывать за 30 секунд. Из-за того, что модель обучается довольно долго, полноценный код процедуру тестирования не пройдёт.

In [ ]:
# Поставим недостающий пакет
!pip3 install sktime --ignore-installed

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats 

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import seaborn as sns
%matplotlib inline

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

## Данные

Данные взяты [с соревнавания на kaggle.](https://www.kaggle.com/neuromusic/avocado-prices) В табличке ниже представлены месячные данные по розничной торговле авокадо:

* `Date` - дата наблюдения
* `AveragePrice` - средняя цена одного авокадо
* `Type` - обычный или органический
* `Year` - год
* `Region` - город или регион наблюдения

Кроме этих колонок есть ещё несколько технических, но они нас не будут интересовать. 

In [ ]:
dff = pd.read_csv('avocado_small.tsv', sep='\t', parse_dates=['Date']).set_index('Date')

# Периодичность данных - месячная 
dff.index = dff.index.to_period("M")

# будем смотреть только на тип organic 
dff = dff[dff.Type == 'organic'] 
print(dff.shape)
dff.head()

Сразу же ответим на вопрос из шапки этой тетрадки и посмотрим как в разных городах цена на авокадо менялась год от года.

In [ ]:
sns.factorplot('AveragePrice', 'Region', data=dff, hue='Year',
                size=13, aspect=0.8, palette='magma', join=False);

Кажется, что Сан-Франциско точно не подходит. Судя по всему, миллениалов может спасти Хьюстон. 

In [ ]:
df = dff[['AveragePrice', 'Region']]
df.head()

In [ ]:
df.index.unique().size # 39 уникальных точек (месячные данные)

In [ ]:
df.Region.unique().size # 54 города

Будем работать с Бостоном. Разобьём выборку на трейн и тест. С помощью функции `plot_series` 

In [ ]:
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split

TEST_SIZE = 12
REGION = 'Boston'

y = df[df['Region'] == REGION].sort_index()['AveragePrice'].asfreq('m')

y_train, y_test = ... , ...

# your code here


In [ ]:
from sktime.forecasting.base import ForecastingHorizon

fh = ForecastingHorizon(y_test.index, is_relative=False)
fh

В качестве метрики качества будем использовать MAPE.

In [ ]:
def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 1. Прогнозы и ансамбль

__[а]__ Постройте наивный прогноз. Выберите из трёх стратегий для строительства наивного прогноза самую лучшую. Качество модели, полученной на тестовом периоде запишите в переменную `ans1`. 

__Примечание:__ Cреднее для наивного прогноза считайте по окну размера`TEST_SIZE`.

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

ans1 = ...

# your code here


In [ ]:
assert ans1  < 12

# несколько похожих скрытых тестов

__[б]__  Используя `sm.tsa.statespace.ExponentialSmoothing` обучите $ETS(AAA)$ модель. Поставьте `seasonal=12`. Постройте доверительный интервал для прогнозов. Изобразите его на картинке. Найдите качество работы модели на тестовой выборке и запишите результат в переменную `ans2`. Удалось ли улучшить наивный прогноз? 

In [ ]:
import statsmodels.api as sm

### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

ans2 = ...

# your code here


Проведите диагностику модели. Правда ли, что остатки модели оказались нормальными? Можно ли использовать её для точечных прогнозов? Можно ли с её помощью строить для прогнозов надёжные доверительные интервалы? 

In [ ]:
# your code here


In [ ]:
assert ans2 < 15

# несколько похожих скрытых тестов

__[в]__  Попробуем новую для нас модель, $TBATS$. Это некоторое улучшение $ETS$, с тем как оно работает, мы будем разбиратться на следующей неделе. Пока просто обучим её и посмотрим на метрику качества. Запишите MAPE модели на тестовой выборке `ans3`. Не забудьте нарисовать картинку с динамикой прогнозов. 

In [ ]:
from sktime.forecasting.tbats import TBATS

forecaster = TBATS(sp=12, 
                   use_trend=True, 
                   use_damped_trend=True,
                   use_arma_errors=True,
                   use_box_cox=True)
    
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you
    
# your code here


In [ ]:
assert ans3 < 10

# несколько похожих скрытых тестов

__[г]__ Попробуем заключительную модель перед ансамблем, тетта-метод. В переменную `ans4` запишите получившеесся MAPE.

In [ ]:
from sktime.forecasting.theta import ThetaForecaster
forecaster = ThetaForecaster(sp=12)

### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you
    
# your code here


In [ ]:
assert ans4 > 18

# несколько похожих скрытых тестов

__[д]__ Объединим все оценённые нами модели в ансамбль. Заполните недостающие части кода. В качестве `ans5` запишите MAPE для ансамбля. Удалось ли с помощью него получить хорошее качество? 

In [ ]:
from sktime.forecasting.compose import EnsembleForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing


### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

forecaster = EnsembleForecaster(
    [("ets",   ExponentialSmoothing(trend="add", seasonal="add", sp=12)),
     # tbats model here
     # theta model here
    ]
)
    

# your code here


In [ ]:
assert ans5 > 10

# несколько похожих скрытых тестов

> __ВАЖНО!!!__ Перед отправкой тетрадки в грейдер на оценивание закомментируйте код, который обучает модели. Оставьте только ответы, записанные в соотвествующие переменные. При выставлении оценки ваш код должен отрабатывать за 30 секунд. Из-за того, что модель обучается довольно долго, полноценный код процедуру тестирования не пройдёт.

# 2. Кросс-валидация

Понятное дело, что просто разбить выборку на трэйн и тест, а затем построить прогноз на всю тестовую часть - мало. Такие оценки качества моделей не очень стабильны. Более того, при прогнозировании на разные горизонты, мы получаем разные ошибки.

Для кросс-валидации будем использовать класс [SlidingWindowSplitter,](https://www.sktime.org/en/latest/api_reference/modules/auto_generated/sktime.forecasting.model_selection.SlidingWindowSplitter.html#sktime.forecasting.model_selection.SlidingWindowSplitter) в рамках которого реализовано скользящее окно. В тестовой выборке будем держать три месяца.

На семинаре мы использовали средства sklearn, но sktime на порядок удобнее. 

In [ ]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

# параметры для кросс-валидации
window_length = 27                     # y_train.size
fh = ForecastingHorizon([1, 2, 3])     # максимум на 3 месяца вперёд
fh_length = len(fh)                    # даты для тестовой выборки

cv_window = SlidingWindowSplitter(window_length=window_length, fh=fh, start_with_window=True)

n_splits = cv_window.get_n_splits(y)
print(f"Number of Folds = {n_splits}")

Выглядеть наше разбиение будет вот таким образом. Голубое - обучающая выборка, оранжевое - тестовая. Найти код для строительства такой картинки можно [в документации.](https://github.com/alan-turing-institute/sktime/blob/master/examples/window_splitters.ipynb) Там же описано много других стратегий кросс-валидации, уже реализованных в рамках пакета. 

![](split.png)

Попробуем провести кросс-валидацию для Тетта-метода. Допишите в функции ниже нехватающие для этого части.

In [ ]:
def mape(y_true, y_pred): 
    return np.abs((y_true - y_pred) / y_true)* 100

def cross_val_sccore(forecaster, y, cv=cv_window):
    M = [ ]

    for idx_train, idx_val in cv.split(y):
        
        y_tr = ...
        y_val = ...
        fh_val = ...
        
        # обучение модели
        
        y_pred = ...  # прогнозирование на период fh_val
        
        # your code here
        
        
        M.append(mape(y_val, y_pred).values)
        
    return np.array(M)

M = cross_val_sccore(ThetaForecaster(sp=12), y)
quality = M.mean(axis=0)
quality

В векторе quality записано три различных значения MAPE нашей модели при прогнозировании на месяц, два и три вперёд.

In [ ]:
assert M.shape == (10, 3) # это не тест на оценку, его можно закоментить при отправке в грейдер

Проведите кросс-валидацию для всех моделей из списка ниже. Какая из них показывает наилучший результат? В переменную `ans6` запишите номер модели, которая лучше всего прогноизрует на месяц вперёд. В перемменную `ans7` запишите номер модели, которая лучше всего прогнозирует на три месяца вперёд. Провизуализируйте получившееся качество на картинке по аналогии с семинаром.

In [ ]:
from sktime.forecasting.ets import AutoETS

forecaster_1 = NaiveForecaster(strategy="last")
forecaster_2 = NaiveForecaster(strategy="mean", window_length=12)
forecaster_3 = NaiveForecaster(strategy="last", sp=12)

forecaster_4 = ThetaForecaster(sp=12)
forecaster_5 = AutoETS(auto=True, sp=12, n_jobs=-1, information_criterion='bic')

forecaster_6 = TBATS(sp=12, 
                   use_trend=True, 
                   use_damped_trend=True,
                   use_arma_errors=True,
                   use_box_cox=True)

ans6 = ... 
ans7 = ...

# your code here


In [ ]:
assert ans6 > 0
assert ans7 > 0
assert ans6 < 7 
assert ans7 < 7

# несколько похожих скрытых тестов

> __ВАЖНО!!!__ Перед отправкой тетрадки в грейдер на оценивание закомментируйте код, который обучает модели. Оставьте только ответы, записанные в соотвествующие переменные. При выставлении оценки ваш код должен отрабатывать за 30 секунд. Из-за того, что модель обучается довольно долго, полноценный код процедуру тестирования не пройдёт.

# Необязательное задание:

1. Попробуйте прогнать наш пайплайн на всех городах. Найдите модель, которая побеждает в качестве прогноза чаще всего.

2. Прочитайте [туториал от sktime,](https://nbviewer.jupyter.org/github/alan-turing-institute/sktime/blob/master/examples/01_forecasting.ipynb) в нём вы найдёте пример того как делать Griidsearch, а также онлайн-прогнозирование с постепенно сдвигающимся окном. Попробуйте добавить в наш пайплайн [другие модели](https://www.sktime.org/en/latest/api_reference.html#sktime-forecasting-time-series-forecasting) и сравнить их между собой. Можно попробовать добавить `Autoarima`, `Prophet`, `KNeighborsRegressor`. Обязательно посстарайтесь разобраться, как именно они строят свой прогноз. Если не выходит, приходите на форум, будем разбираться вместе! 

2. На гите библиотеки [есть довольно много примеров кода.](https://github.com/alan-turing-institute/sktime/tree/master/examples)

 ## Разбираемся в теории:
 
В этом задании вы встретились с таким алгоритмом, как Тетта-метод. Его придумали в $2000$ году и он довольно простой. Попробуйте [самостоятельно разобраться с ним](https://robjhyndman.com/papers/Theta.pdf) по аналогии с тем, как мы на лекциии разбирали другие модели: 
 
 1. Выпишите уравнение модели
 2. Разбиритесь как строится точечный прогноз
 3. Разберитессь как строится доверительный интервал 
 4. Придумайте простую задачку с цифрами, из 5 наблюдений, и решите её на листочке

Не пугайтесь формул из статьи! Они и правда могут показаться страшными. 

<center>
<img src="https://pbs.twimg.com/media/DZ4kgtMWkAAhcyb.jpg" width="700"> 
</center>